In [ ]:
# os is used for Ilka's linux :)
#import os
#os.environ["QT_QPA_PLATFORM"] = "xcb"

import cv2
from ultralytics import YOLO


In [ ]:
from ultralytics import YOLO

# Load a model
# model = YOLO("yolo11n.yaml")  # build a new model from YAML
# model = YOLO("yolo11n.pt")  # load a pretrained model (recommended for training)
model = YOLO("yolo11n.yaml").load("yolo11n.pt")  # build from YAML and transfer weights

# Train the model
results = model.train(data="./dataset.yaml", epochs=50, imgsz=640, batch=4)


and now predict

In [4]:
from ultralytics import YOLO
import cv2

# Load your trained YOLO model
model = YOLO("./runs/detect/50-epochs/weights/best.pt")  

# Open the video file
video_path = "/home/jeski/sem/vc/test-vid-prof.mp4"  # Replace with your video file path
cap = cv2.VideoCapture(video_path)

# Check if video opened successfully
if not cap.isOpened():
    print("Error: Could not open video.")
    exit()

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Set up video writer to save output (optional)
output_path = "/home/jeski/sem/vc/test-vid-prof-50_epochs-track.mp4"  # Replace with desired output path
fourcc = cv2.VideoWriter_fourcc(*"XVID")
out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))

# Process each frame of the video
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO model on the frame
    results = model(frame)
    
    # Visualize the results on the frame
    annotated_frame = results[0].plot()  # plot() adds bounding boxes and labels to the frame

    # Write the annotated frame to the output video
    out.write(annotated_frame)

# Release resources
cap.release()
out.release()
print(f"Processed video saved to {output_path}")


OpenCV: FFMPEG: tag 0x44495658/'XVID' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'



0: 384x640 (no detections), 85.3ms
Speed: 4.2ms preprocess, 85.3ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 100.9ms
Speed: 2.9ms preprocess, 100.9ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 109.6ms
Speed: 5.4ms preprocess, 109.6ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 87.1ms
Speed: 12.2ms preprocess, 87.1ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 95.4ms
Speed: 2.7ms preprocess, 95.4ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 license plate, 81.6ms
Speed: 2.8ms preprocess, 81.6ms inference, 1.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 77.6ms
Speed: 6.7ms preprocess, 77.6ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 111.4ms
Speed: 2.8ms preprocess, 11

KeyboardInterrupt: 

test with tracking


In [3]:
import cv2
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO("/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/runs/detect/50-epochs/weights/best.pt")  # Replace with your model path

# Specify the input and output video paths
video_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof.mp4"  # Replace with your input video path
output_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof.mp4"  # Path to save the annotated video

# Open the input video file
cap = cv2.VideoCapture(video_path)

# Get video properties for output video
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))

# Define the codec and create VideoWriter object
out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*"mp4v"), fps, (frame_width, frame_height))

# Iterate through each frame in the video for tracking
for results in model.track(source=video_path, stream=True):
    # Get the annotated frame
    annotated_frame = results.plot()  # Adds bounding boxes and labels to the frame

    # Write the annotated frame to the output video
    out.write(annotated_frame)

# Release the resources
cap.release()
out.release()

print(f"Annotated video saved to {output_path}")




WARNING ⚠️ inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/2832) /home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof.mp4: 384x640 (no detections), 88.5ms
video 1/1 (frame 2/2832) /home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof.mp4: 384x640 1 license plate, 72.6ms
video 1/1 (frame 3/2832) /home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-vid-prof.mp4: 384x640 1 license plate, 192.5ms
video 1/1 (frame 4/2832) /home/jeski/sem/vc/Visi-n-por-Computador/practica

KeyboardInterrupt: 

test with image


In [3]:
import cv2
from ultralytics import YOLO

# Load the trained YOLO model
model = YOLO("/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/runs/detect/train/weights/best.pt")  # Replace with the path to your trained model

# Load the image
image_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/Our-Images/train/images/1729779190271.jpg"  # Replace with the path to your image
image = cv2.imread(image_path)

# Check if the image was loaded successfully
if image is None:
    print("Error: Could not load image.")
    exit()

# Run YOLO model on the image
results = model(image)

# Visualize the results on the image
annotated_image = results[0].plot()  # plot() adds bounding boxes and labels to the image

# Display the result
#cv2.imshow("YOLO Detection", annotated_image)

# Wait for a key press to close the window
#cv2.waitKey(0)
#cv2.destroyAllWindows()

# Save the result (optional)
output_path = "/home/jeski/sem/vc/Visi-n-por-Computador/practica4/test-1.jpg"  # Specify where to save the result
cv2.imwrite(output_path, annotated_image)
print(f"Annotated image saved to {output_path}")


FileNotFoundError: [Errno 2] No such file or directory: '/home/jeski/sem/vc/Visi-n-por-Computador/practica4/datasets/selfmadeYOLO/runs/detect/train/weights/best.pt'